### LLaMa에 전이학습

In [1]:
# trl(transformers reinforecement learning) : 강화학습을 통해 트랜스포머 언어모델을 미세조정
# peft(parameter efficient find tunning) : 대규모 언어 모델을 효율적으로 미세 조정 

!pip install -q datasets bitsandbytes trl peft accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 26.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
import torch
from datasets import load_dataset
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

In [3]:
# GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
# 한글 wikipedia 교육용 사용
login("________")

In [5]:
# 데이터셋 로드
dataset = load_dataset("devngho/korean-wikipedia-edu")

README.md:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

scored_over_3/train-00000-of-00001.parqu(…):   0%|          | 0.00/104M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65161 [00:00<?, ? examples/s]

In [6]:
# 양자와 설정 : Kaggle에서 안정성을 위한 double_quant 사용
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)


In [ ]:
# LLaMa3 Token
token = "_________"
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
import wandb, os
wandb.login(key="_______")
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zeushahn (zeushahn-khankong) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [9]:
# 토크나이저 
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=token
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
# 모델 로드 
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map='auto',
    token=token,
    dtype=torch.float16
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

2025-09-20 02:44:57.289520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758336297.519978      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758336297.584801      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [11]:
# 패딩 토큰 설정
tokenizer.pad_token = tokenizer.eos_token
model.config.use_cache = False

In [12]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'score'],
        num_rows: 65161
    })
})


In [13]:
print(dataset['train']['text'][0])

안토니오 그리마니의 문장.
**안토니오 그리마니**(1434년 12월 28일 -- 1523년 5월 7일)는 1521년부터 1523년까지의 베네치아 공화국의 도제이다.
## 생애
그는 베네치아에서 상대적으로 가난한 집안에서 태어났고 소매상인으로서 몇 년을 종사하다 곧 베네치아 내에서 가장 핵심적인 인물 중 한 명이 되었다. 1494년에 그는 베네치아 총사령관으로 임명되어, 평화로운 기간에 아드리아해 연안을 따라 베네치아의 작전을 이끌어야했기 때문에 상대적으로 쉬운 일이었다. 1499년에 베네치아와 오스만 튀르크 제국 사이의 새로운 전쟁이 발발하였으며, 그의 경험 부족은 두 차례의 패배(8월 19일 사피엔차, 6일 뒤인 존키오 )로 이어졌다.
그리마니는 사형 놓이게 됐지만 크레스 섬 추방으로 바뀌었다. 하지만 곧 그는 1509년에 로마로 망명했다.
그의 아들들의 중재덕에 1509년 베네치아로 돌아갈 수 있었다. 그의 정치적 유대 관계는 곧 그를 가장 핵심적인 행정 자리를 얻게 했고 미래의 도제 자리(1521년 7월 6일)를 만드는 기반을 만들었다.
그가 도제 자리에 오르자, 공화국을 1521년 이탈리아 전쟁으로 이끌었고, 프랑수아 1세의 유일한 동맹이였던 그는 그를 버리지 않았다. 그러나 프랑스가 비코카 전투에서 패한 후, 그는 전쟁의 과정에 대하여 염려했고, 그러나 그가 1523년에 사망하면서 그의 후임인 안드레아 그리티는 카를 5세와의 강화 조약을 맺었다.
그는 카테리나 로레단(Caterina Loredan)과 혼인했다. 그의 아들인 도메니코 그리마니는 1493년에 추기경이 되었다.


### LoraConfig
: LoRa기법을 사용해 모델에 미세 조정할때 필요한 설정을 정의하는 클래스 

In [14]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=128,
    lora_alpha=4,
    lora_dropout=0.1,
    task_type='CAUSAL_LM'
)

### SFT Trainer를 통한 미세 조정

In [15]:
from trl import SFTTrainer, SFTConfig

In [16]:
training_args = SFTConfig(
    output_dir = ".",
    per_device_train_batch_size=1, #epoch
    gradient_accumulation_steps = 5, 
    learning_rate=2e-4,
    max_steps=500,
    warmup_steps=100,
    logging_steps=100,
    fp16=True,
    optim="paged_adamw_8bit",
    seed=42,
    max_length=128,
    dataset_text_field= 'text'
)

In [17]:
# Trainder 설정

trainer = SFTTrainer(
    model = model,
    processing_class = tokenizer, # tokenizer = tokenizer
    args = training_args,
    peft_config = peft_config,
    train_dataset=dataset['train']
)

Adding EOS to train dataset:   0%|          | 0/65161 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/65161 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/65161 [00:00<?, ? examples/s]